In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [10]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
filtered_df = application_df.drop(columns=['EIN', 'NAME'])
filtered_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:
filtered_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [12]:
small = ['0', '1-9999', '10000-24999']
medium = ['25000-99999', '100000-499999']
large = ['1M-5M', '5M-10M','10M-50M', '50M+']

for company in small:
    filtered_df['INCOME_AMT'] = filtered_df['INCOME_AMT'].replace(company,"SMALL")

for company in medium:
    filtered_df['INCOME_AMT'] = filtered_df['INCOME_AMT'].replace(company,"MEDIUM")

for company in large:
    filtered_df['INCOME_AMT'] = filtered_df['INCOME_AMT'].replace(company,"LARGE")

filtered_df['INCOME_AMT'].value_counts()

SMALL     25659
MEDIUM     7121
LARGE      1519
Name: INCOME_AMT, dtype: int64

In [13]:
filtered_df = filtered_df.loc[filtered_df['USE_CASE'] != 'Other', :]
filtered_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Name: USE_CASE, dtype: int64

In [14]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
rare_app = ['T9', 'T13', 'T12']
ultra_rare_app = ['T2', 'T25', 'T14', 'T29', 'T15', 'T17']
#'T10', 'T9', 'T13', 'T12'
# Replace in dataframe
for app in rare_app:
    filtered_df['APPLICATION_TYPE'] = filtered_df['APPLICATION_TYPE'].replace(app,"RARE")

for app in ultra_rare_app:
    filtered_df['APPLICATION_TYPE'] = filtered_df['APPLICATION_TYPE'].replace(app,"ULTRA_RARE")

# Check to make sure binning was successful
filtered_df['APPLICATION_TYPE'].value_counts()

T3            27035
T4             1542
T6             1216
T5             1173
T19            1065
T8              736
T7              725
T10             528
RARE            249
ULTRA_RARE       27
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Look at CLASSIFICATION value counts for binning
classifications = filtered_df['CLASSIFICATION'].value_counts()
cls_df = pd.DataFrame(classifications)
cls_df

,CLASSIFICATION
C1000,17323
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1
C4500,1


In [16]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for idx, cls in cls_df.iterrows():
    if cls[0] < 10:
        classifications_to_replace.append(idx)

# Replace in dataframe
for cls in classifications_to_replace:
    filtered_df['CLASSIFICATION'] = filtered_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
filtered_df['CLASSIFICATION'].value_counts()

C1000    17323
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(filtered_df)
dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_RARE,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_LARGE,INCOME_AMT_MEDIUM,INCOME_AMT_SMALL,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,1,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,1,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,1,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,1,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,1,1,0


In [18]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL']
X = dummies_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2 = 5
#hidden_nodes_layer3 = 3
#hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="elu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="elu"))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="elu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 9)                 558       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 50        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 614
Trainable params: 614
Non-trainable params: 0
_________________________________________________________________


In [21]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=30)

Epoch 1/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5990 - accuracy: 0.7072: 0s - loss: 0.6096 - accura
Epoch 2/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5632 - accuracy: 0.7261
Epoch 3/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5576 - accuracy: 0.7293: 0s - loss: 0
Epoch 4/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7301
Epoch 5/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7292
Epoch 6/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7289
Epoch 7/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7305: 0s - loss: 0.5491 - accuracy: 
Epoch 8/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7312
Epoch 9/30
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - accuracy: 0.7310
Epoch 10/30
804/804 [=====

In [23]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5532 - accuracy: 0.7266
Loss: 0.5531504154205322, Accuracy: 0.7266153693199158
